In [ ]:
#Python Codes for Grey Wolf Optimization Algorithm
#step 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import time
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
#step 2 loading data set
ds_tar1=pd.read_csv('d:/msk/jack data/target_all.csv') # Table 1: Target Dataset of 115 MFL Crack Images
ds_fea=pd.read_csv('d:/msk/jack data/all_features.csv') # Table 2: Features Dataset of 115 MFL Crack Images
ds_tar = ds_tar1.iloc[:,0] #for length change 1 to depth and 2 to width
#step 3 Greywolf optimization algorithm 
fea=pd.DataFrame()
n_sample = 50 #number of wolves
nof = 17 #no. of features change this between 15 and 22
nitr = 100 #number of iterations
fnc1=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21','f22']#feature column names
fnc = fnc1[:nof]
trial = 'T1' #trail number
ps_tr = []
rs_tr = []
ps_tt = []
rs_tt = []
so = []
feano = pd.DataFrame()
mdl = XGBRegressor()
#population initialozation and calculation of objective function
for i in range(n_sample):
    print('Sample No.',i)
    fea[i]=pd.Series(random.sample(range(0, 22), nof)) 
feat = fea.T
for i in range(fea.shape[1]):
    for j in range(fea.shape[0]):
        feat.iloc[i,j] = fea.iloc[j,i]
#evaluation of population
for i in range(fea.shape[1]):
    ds_sfea = ds_fea.iloc[:,fea[i]]
    xtr, xtt, ytr, ytt = train_test_split(ds_sfea, ds_tar, test_size = 0.2, random_state = 2)
    mdl.fit(xtr, ytr)
    ytr_p = mdl.predict(xtr)
    ytt_p = mdl.predict(xtt)
    rs_tr.append(r2_score(ytr, ytr_p))
    ps_tr.append(np.sqrt(mean_squared_error(ytr, ytr_p)))
    rs_tt.append(r2_score(ytt, ytt_p))
    ps_tt.append(np.sqrt(mean_squared_error(ytt, ytt_p)))
    
pstr = pd.Series(ps_tr)
pstt = pd.Series(ps_tt)
rstr = pd.Series(rs_tr)
rstt = pd.Series(rs_tt)
per = pd.concat([pstr, pstt, rstr, rstt], axis=1)
per.columns=['pstr','pstt','rstr','rstt']
feat.columns=fnc
so = pd.concat([per, feat], axis = 1)
so['Obj']=so.iloc[:,0]*0.25 + so.iloc[:,1]*0.25 + so.iloc[:,2]*0.25 + so.iloc[:,3]*0.25
so[fnc]=so[fnc].astype('int64')
so1 = so
so1[fnc]=so1[fnc].astype('int64')
#selecting the best one
so1.sort_values('Obj', ascending = False)
bst = so1.iloc[0,:]
rslt = pd.DataFrame()
rs =[]
for i in range(bst.shape[0]):
    rs.append(bst[i])
rslt[0] = pd.Series(rs)
nso = so;
#initialization of GWO parameters
sf=2 #scaling factor
als = 0
bes = 0
des = 0
s1=[]
s2=[]
s3=[]
for it in range(nitr):
    print('Iteration ',it,'\nPopulation:',so)
    for i in range(n_sample):
        if so.iloc[i,-1] > als:
            als = so.iloc[i,-1]
            alp = so.iloc[i,4:-1]
            alo = so.iloc[i,-1]
        if so.iloc[i,-1] < als and so.iloc[i,-1] > bes:
            bes = so.iloc[i,-1]
            bep = so.iloc[i,4:-1]
            beo = so.iloc[i,-1]
        if so.iloc[i,-1] < als and so.iloc[i,-1] < bes and so.iloc[i,-1] > des:
            des = so.iloc[i,-1]
            dep = so.iloc[i,4:-1]
            deo = so.iloc[i,-1]  
    a = sf - it *(sf/nitr) #a decreasing linearly from 2 to 0
    for i in range(n_sample):
        pos = so.iloc[i,4:-1]
        for j in range(nof):
            r1=random.random()
            r2 = random.random()
            A1 = 2*a*r1-a
            C1 = 2*r2
            D_al = np.abs(C1*alp[j]- pos[j])
            X1 = alp[j]-A1*D_al
            r3 = random.random()
            r4 = random.random()
            A2=2*a*r3-a
            C2=2*r4
            D_be = np.abs(C2*bep[j]- pos[j])
            X2 = bep[j]-A2*D_be
            r5 = random.random()
            r6 = random.random()
            A3=2*a*r5-a
            C3=2*r6
            D_de = np.abs(C3*dep[j]- pos[j])
            X3 = dep[j]-A3*D_de
            pos[j] = (X1+X2+X3)/3
            if pos[j] > 21: #position greater than number of feature 
                pos[j] = 21
            if pos[j] < 0: #position less than zero
                pos[j]=0
        npo = pd.Series(pos)      
        npo.columns = fnc              
        npo[fnc]=npo[fnc].astype('int64')
                
        s1=[]
        s2=[]
        s3=[]
        s1=npo
        s2=[*set(s1)]               
        [s3.append(x) for x in list(range(22)) if x not in s2]
        len1 = nof - len(s2)
        for i1 in range(len1):
            s2.append(s3[i1])               
        npo=pd.Series(s2)        
        ds_sfean = ds_fea.iloc[:,s2]
        xtr1, xtt1, ytr1, ytt1 = train_test_split(ds_sfean, ds_tar, test_size = 0.2, random_state = 1)
        mdl.fit(xtr1, ytr1)
        ytr_p1 = mdl.predict(xtr1)
        ytt_p1 = mdl.predict(xtt1)
        rs_tr1=(r2_score(ytr1, ytr_p1))
        ps_tr1=(np.sqrt(mean_squared_error(ytr1, ytr_p1)))
        rs_tt1=(r2_score(ytt1, ytt_p1))
        ps_tt1=(np.sqrt(mean_squared_error(ytt1, ytt_p1)))
        obj1 = ps_tr1*0.25 + ps_tt1 *0.25 + rs_tr1*0.25 + rs_tt1 *0.25       
        nso.iloc[i,4:-1] = npo
        nper = pd.Series([ps_tr1,ps_tt1,rs_tr1,rs_tt1])
        nso.iloc[i,0:4] = nper
        nso.iloc[i,-1] = obj1
        
    nso1 = nso
    nso1.sort_values('Obj',ascending = False)
    bst1 = nso1.iloc[0,:]
    rs1=[]
    for i2 in range(bst1.shape[0]):
        rs1.append(bst1[i2])
    rslt[it+1] = pd.Series(rs1)
    so = nso
####convergence plot
rslt1 = rslt.T
rslt1=pd.DataFrame(rslt1)
fna = 'd:/msk/JAck Data/gwo_lir_jack_nof' + str(nof) + '181222_25523.xlsx'
rslt1.to_excel(fna)
print(rslt1.shape[0])
ptr=[rslt1.iloc[0,0]]
ptt=[rslt1.iloc[0,1]]
rtr=[rslt1.iloc[0,2]]
rtt=[rslt1.iloc[0,3]]
sobj=[rslt1.iloc[0,-1]]
for i in range(rslt1.shape[0]):
    if rslt1.iloc[i,0] > ptr[i]:
        ptr.append(rslt1.iloc[i,0])
    else:
        ptr.append(ptr[i])
    if rslt1.iloc[i,1] > ptt[i]:
        ptt.append(rslt1.iloc[i,1])
    else:
        ptt.append(ptt[i])
    if rslt1.iloc[i,2] > rtr[i]:
        rtr.append(rslt1.iloc[i,2])
    else:
        rtr.append(rtr[i])
    if rslt1.iloc[i,3] > rtt[i]:
        rtt.append(rslt1.iloc[i,3])
    else:
        rtt.append(rtt[i])
    if rslt1.iloc[i,-1] > sobj[i]:
        sobj.append(rslt1.iloc[i,-1])
    else:
        sobj.append(sobj[i])        
x = range(len(ptr))
f1, ax1 = plt.subplots()
f2, ax2 = plt.subplots()
f3, ax3 = plt.subplots()
ax1.plot(x, ptr,'g-')
ax1.plot(x, ptt,'b-')
ax1.set_xlabel('Iteration Number')
ax1.set_ylabel('RMSE')
ax1.legend(['Training Dataset', 'Testing Dataset'])
ax2.plot(x, np.abs(rtr),'r-')
ax2.plot(x, np.abs(rtt),'m-')
ax2.set_xlabel('Iteration Number')
ax2.set_ylabel('${R{^2}}$')
ax2.legend(['Training Dataset', 'Testing Dataset'])    
ax3.plot(x, sobj, color='g')
ax3.set_xlabel('Iteration Number')
ax3.set_ylabel('Objective Value')
